# Car Price prediction

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

# Прогнозирование стоимости автомобиля по характеристикам

## Установка и импорт библиотек:

In [ ]:
# использование этой библиотеки связано со сложностями, приходится устанавливать ее, перезапускать Kernel,
# и только тогда запускать остальной код. Тогда все работает
#!pip install lazypredict

In [ ]:
!pip install - q tensorflow == 2.3

In [ ]:
# аугментации изображений
!pip install albumentations - q

In [ ]:
from pylab import rcParams
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import string


from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from pandas_profiling import ProfileReport
from lazypredict.Supervised import LazyRegressor
from tqdm.notebook import tqdm

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations as A

# nltk
import nltk
nltk.download('stopwords')

# plt
# увеличим дефолтный размер графиков
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
# фиксируем RANDOM_SEED
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
train.nunique()

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [ ]:
# split данных
data_train, data_test = train_test_split(
    train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

# оцениваем точность
print(
    f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

Проведем быстрый анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1
test['sample'] = 0
test['price'] = 0

data = test.append(train, sort=False).reset_index(drop=True)

In [ ]:
# Для быстрой оценки используем pandas_profiling
# ProfileReport(data)

### Получаем следующий перечень признаков:
 - bodyType, тип кузова (категориальный). Поскольку у нас уже имеются данные о кол-ве дверей, данный столбец можно от этой информации избавить, уменьшив при этом cardinality;
 - brand, марка автомобиля (категориальный). Представлен всего тремя видами, в обработке не нуждается;
 - color, цвет автомобиля (категориальный). 11 видов, в обработке не нуждается;
 - description, текстовый. Требует обработки нейросетью для использования;
 - engineDisplacement, объем двигателя (категориальный). Можно убрать сочетание LTR и переделать признак в числовой. Пока непонятно, есть ли в этом необходимость;
 - enginePower, мощность двигателя (категориальный). Также можно убрать N12 и переделать в числовой. Может помочь избавиться от HIGH CARDINALITY;
 - fuelType, вид топлива (категориальный). 4 вида, в обработке не нуждается;
 - mileage, пробег (числовой). Чрезвычайно много значений в начале. Требуется нормализация;
 - modelDate, год выпуска модели (числовой). Много значений в конце. Требуется нормализация;
 - model_info, название модели (категориальный). Много уникальных значений;
 - name, составной признак. Для различных записей содержит информацию о: названии модели, объеме двигателя, его мощности в лошадиных силах, типе коробки, наличии полного привода. Все эти данные имеются в других колонках, данный признак можно удалить;
 - numberOfDoors, кол-во дверей (категориальная). Обработки не требует;
 - productionDate, год производства автомобиля (числовой). Нормализуем;
 - sell_id, идентификационный номер. Для построения модели не используется;
 - vehicleConfiguration, составной признак. Может содержать информацию о: типе кузова, кол-ве дверей, типе коробки, объеме двигателя. Поскольку в других колонках уже содержится вся эта информация, признак можно удалить;
 - vehicleTransmission, тип КПП (категориальный). 4 вида значений, обработки не требует.
 - Владельцы (категориальный). Можно убрать текст;
 - Владение, сколько находится в собственности последнего владельца. Огромное кол-во пропусков (более 64%). Скорее всего, данный признак придется удалить;
 - ПТС (категориальный). 2 вида значений, в обработке не нуждается;
 - Привод (категориальный). 3 вида значений, в обработке не нуждается;
 - Руль (категориальный). 2 вида значений, но одного из них настолько мало, что признак использовать не получится;
 - price, стоимость автомобиля (числовой). Целевой признак, который нужно предсказать.

# PreProc Tabular Data

In [ ]:
# разделяем все признаки на категориальные и числовые
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 'numberOfDoors',
                        'vehicleTransmission',
                        'ПТС',
                        'Привод']

numerical_features = ['mileage', 'modelDateNorm', 'productionDateNorm', 'engineDisplacement',
                      'enginePower', 'Владельцы',
                      # 'Устаревание',
                      'year_mileage'
                      ]

In [ ]:
def preproc_data(df_input, train=0, lazy=0):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()

    # ################### 1. Предобработка ##############################################################
    # убираем не нужные для модели признаки
    df_output.drop(['Руль', 'Владение', 'sell_id', 'vehicleConfiguration',
                   'name', 'description'], axis=1, inplace=True)

    # убираем дупликаты строк
    # df_output.drop_duplicates(inplace=True)

    # генерация фич
    # Устаревание
    #df_output['Устаревание'] = df_output.productionDate-df_output.modelDate

    # Ежегодный пробег
    df_output['year_mileage'] = df_output.mileage / \
        (2021-df_output.productionDate)

    # ################### Numerical Features ##############################################################
    # Далее заполняем пропуски
    # for column in numerical_features:
    #df_output[column].fillna(df_output[column].median(), inplace=True)

    # тут ваш код по обработке NAN
    df_output['Владельцы'].fillna(
        df_output['Владельцы'].mode()[0], inplace=True)
    df_output.engineDisplacement[df_output.engineDisplacement ==
                                 'undefined LTR'] = df_output.engineDisplacement.mode()[0]

    #  Переводим признаки в числовые
    df_output.engineDisplacement = df_output.engineDisplacement.apply(
        lambda x: x.split()[0])
    df_output.enginePower = df_output.enginePower.apply(lambda x: x.split()[0])
    df_output['Владельцы'] = df_output['Владельцы'].apply(
        lambda x: x.split()[0])

    df_output.enginePower = df_output.enginePower.apply(lambda x: int(x))
    df_output.engineDisplacement = df_output.engineDisplacement.apply(
        lambda x: float(x))
    df_output['Владельцы'] = df_output['Владельцы'].apply(lambda x: int(x))

    # логарифмируем где нужно
    df_output['modelDateNorm'] = np.log(2021 - df_output['modelDate'])
    df_output['productionDateNorm'] = np.log(
        2021 - df_output['productionDate'])

    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    # ################### Categorical Features ##############################################################
    # bodyType - оставляем только тип кузова
    df_output.bodyType = df_output.bodyType.apply(lambda x: x.split()[0])

    # Label Encoding
    if lazy == 0:
        for column in categorical_features:
            df_output[column] = df_output[column].astype('category').cat.codes

    # One-Hot Encoding; для трейна отключаем, чтобы оценить важность фич
    if (train == 0) and (lazy == 0):
        df_output = pd.get_dummies(
            df_output, columns=categorical_features, dummy_na=False)

    # удаляем лишнее
    df_output.drop(['modelDate', 'productionDate', ], axis=1, inplace=True)

    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
# df_preproc.sample(10)

In [ ]:
# Посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(2, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize=(columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins=20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width=(bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()


visualize_distributions({
    'mileage': data['mileage'],
    'log_mileage': data['mileage'].apply(np.log),

    'modelDate': data['modelDate'],
    'log_modelDate': df_preproc['modelDateNorm'],

    'productionDate': data['productionDate'],
    'log_productionDate': df_preproc['productionDateNorm'],

    'engineDisplacement': df_preproc['engineDisplacement'],
    'enginePower': df_preproc['enginePower'],

    'Владельцы': df_preproc['Владельцы'],
    # 'Устаревание' : df_preproc['Устаревание'],
    'year_mileage': df_preproc['year_mileage'],
})

#### Логарифмирование улучшило распределения для modelDate и productionDate

In [ ]:
# оценим важность численных переменных
train_preproc = preproc_data(train, 1)

imp_num = pd.Series(f_classif(train_preproc[numerical_features], train_preproc.price)[
                    0], index=numerical_features)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

In [ ]:
# оценим важность категориальных переменных

imp_cat = pd.Series(mutual_info_classif(train_preproc[categorical_features], train_preproc.price,
                                        discrete_features=True), index=categorical_features)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

In [ ]:
df_preproc.info()

## Split data

In [ ]:
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)


# предыдущая работа показала, что логарифмирование price
# улучшает предсказание Catboost

y = train_data.price.apply(np.log).values     # логарифмированный таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
#reg = LazyRegressor(predictions=True, custom_metric = mape)
#models, predictions = reg.fit(X_train, X_test, y_train, y_test)

In [ ]:
# ради интереса был использован LazyRegressor. К сожалению, как оказалось, в одном из последних обновлений
# из списка был исключен CatBoostRegressor, так что оценить разницу по сравнению с лучшим здесь, ExtraTrees,
# не получилось. Но CatBoost еще в прошлой работе показал хорошие результаты, так что используем его
# print(models)

# Model 2: CatBoostRegressor

In [ ]:
model = CatBoostRegressor(iterations=5000,
                          # depth=5,
                          learning_rate=0.05,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          # task_type='GPU',
                          )
model.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          verbose_eval=100,
          use_best_model=True,
          plot=True
          )

In [ ]:
test_predict_catboost = model.predict(X_test)
print(
    f"TEST mape: {(mape(np.exp(y_test), np.exp(test_predict_catboost)))*100:0.2f}%")

#### Функция Catboost

In [ ]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations=5000,
                              # depth=5,
                              learning_rate=0.05,
                              random_seed=RANDOM_SEED,
                              eval_metric='MAPE',
                              custom_metric=['RMSE', 'MAE'],
                              od_wait=500,
                              # task_type='GPU',
                              )
    model.fit(X_train, y_train,
              #cat_features = categorical_features,
              eval_set=(X_test, y_test),
              verbose_eval=100,
              use_best_model=True,
              plot=True
              )
    return(model)

#### K-Fold

In [ ]:
# куда пишем предикты по каждой модели
submissions_cat = pd.DataFrame(
    0, columns=["sub_1"], index=sample_submission.index)
score_ls = []
splits = list(KFold(n_splits=5, shuffle=True,
              random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=5,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(np.exp(y_test), np.exp(test_predict))
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(np.exp(y_test), np.exp(test_predict)):0.3f}")
    # submissions
    submissions_cat[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')

print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

#### K-Fold c 2 доп. признаками - 11,3

#### K-Fold без Устаревания - 11,1

#### K-Fold без обоих доп. признаков - 11,3

#### K-Fold без обоих доп. признаков и ПТС - 11,2

#### K-Fold без Устаревания и ПТС - 11,3

#### Наилучший результат - K-Fold без Устаревания

### Submission

In [ ]:
submissions_cat['blend'] = (submissions_cat.sum(
    axis=1))/len(submissions_cat.columns)
sample_submission['price'] = np.exp(submissions_cat['blend'].values)
sample_submission.to_csv('submission_blend_v3.csv', index=False)
sample_submission.head(10)

#### Использование CatBoost оправдалось. На лидерборде один K-Fold Catboost (без нейросетей вообще) показал результат 11,90

# Model 3: Tabular NN

Построим обычную сеть:

In [ ]:
# Добавление Устаревания в нейросеть дает более хорошие результаты
scaler = MinMaxScaler()
df_preproc['Устаревание'] = data.productionDate-data.modelDate
df_preproc['Устаревание'] = scaler.fit_transform(
    df_preproc[['Устаревание']])[:, 0]

In [ ]:
df_preproc

In [ ]:
# В отличие от Catboost, нейросеть работает с логарифмированным таргетом хуже, чем с обычным
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

## Simple Dense NN

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
model.summary()

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### Fit

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500,  # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                    )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

#### mape 11.73% без Устаревания

#### mape 11.63% с Устареванием

In [ ]:
sub_predict_nn1 = model.predict(X_sub)

In [ ]:
sample_submission['price'] = sub_predict_nn1[:, 0]
sample_submission.to_csv('nn1_2_submission.csv', index=False)
sample_submission.head(10)

In [ ]:
blend_sub_predict_1 = (
    np.exp(submissions_cat['blend'].values) + sub_predict_nn1[:, 0]) / 2
sample_submission['price'] = blend_sub_predict_1
sample_submission.to_csv('blend_submission_1.csv', index=False)
sample_submission.head(10)

#### Blend предсказаний K-Fold Catboost и NN показывает mape 11,30 на лидерборде

# Model 4: NLP + Multiple Inputs

In [ ]:
stop_words = stopwords.words("russian")

In [ ]:
data.description

In [ ]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [ ]:
REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;]!.')
BAD_SYMBOLS = re.compile('[0-9+_]')
MIN_STRING_LEN = 3


def clean_text(description):

    description = description.lower()
    description = REPLACE_BY_SPACE.sub(' ', description)
    description = BAD_SYMBOLS.sub('', description)

    description = ' '.join(word for word in description.split() if (
        (word not in stop_words) & (len(word) >= MIN_STRING_LEN)))

    return description

In [ ]:
data['clean_description'] = data.description.apply(clean_text)

In [ ]:
# split данных
text_train = data.clean_description.iloc[X_train.index]
text_test = data.clean_description.iloc[X_test.index]
text_sub = data.clean_description.iloc[X_sub.index]

### Tokenizer

In [ ]:
% % time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.clean_description)

In [ ]:
# tokenize.word_index

In [ ]:
% % time
text_train_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape,
      text_test_sequences.shape, text_sub_sequences.shape, )

In [ ]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

### RNN NLP

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

### MLP

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

### Multiple Inputs NN

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
model.summary()

### Fit

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500,  # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                    )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:, 0]
sample_submission.to_csv('nn2_2_submission.csv', index=False)

In [ ]:
blend_sub_predict_2 = (
    np.exp(submissions_cat['blend'].values) + sub_predict_nn2[:, 0]) / 2
sample_submission['price'] = blend_sub_predict_2
sample_submission.to_csv('blend_submission_2.csv', index=False)
sample_submission.head(10)

#### Blend предсказаний K-Fold Catboost и Multiple inputs NN показывает mape 11,41 на лидерборде, что немного хуже, чем предыдущий вариант. Возможно, проведена неправильная/недостаточная обработка текста

# Model 5: Добавляем картинки

### Data

In [ ]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize=(12, 8))

random_image = train.sample(n=9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
size = (320, 240)


def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)


images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

### albumentations

In [ ]:
transform = A.Compose([

    A.HueSaturationValue(always_apply=False, p=0.2, hue_shift_limit=(-20, 20),
                         sat_shift_limit=(-30, 30), val_shift_limit=(-20, 20)),
    A.Blur(always_apply=False, p=0.2, blur_limit=(3, 9)),
    A.CLAHE(always_apply=False, p=0.2, clip_limit=(
        1, 10), tile_grid_size=(8, 8)),
    A.ChannelShuffle(always_apply=False, p=0.2),
    A.Equalize(always_apply=False, p=0.2, mode='cv', by_channels=True),
    A.GaussNoise(always_apply=False, p=0.2, var_limit=(10.0, 80.0)),
    A.ISONoise(always_apply=False, p=0.2, intensity=(
        0.1, 0.5), color_shift=(0.01, 0.05)),
    A.ImageCompression(always_apply=False, p=0.2, quality_lower=60,
                       quality_upper=100, compression_type=0),
    A.JpegCompression(always_apply=False, p=0.2,
                      quality_lower=80, quality_upper=100),
    A.RGBShift(always_apply=False, p=0.2, r_shift_limit=(-20, 20),
               g_shift_limit=(-20, 20), b_shift_limit=(-20, 20))

])

In [ ]:
# проверяем выбранные модификации изображений, оцениваем их длительность
# длительность приемлемая, изображения узнаваемые
plt.figure(figsize=(12, 8))
for i in range(9):
    %time
    img = transform(image=images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
def make_augmentations(images):
    print('применение аугментаций', end='')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end='')

    augment_dict = transform(image=images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
    print('')

    return augmented_images

In [ ]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.clean_description)

In [ ]:
def process_image(image):
    return transform(image=image.numpy())['image']


def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen=MAX_SEQUENCE_LENGTH)


def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]


def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image, ] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price


def tf_process_val_dataset_element(image, table_data, text, price):
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price


train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.clean_description.iloc[X_train.index], y_train
)).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.clean_description.iloc[X_test.index], y_test
)).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.clean_description.iloc[X_sub.index], y_sub
)).map(tf_process_val_dataset_element)

# проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__()
test_dataset.__iter__().__next__()
sub_dataset.__iter__().__next__()

### Строим сверточную сеть для анализа изображений без "головы"

In [ ]:
# модель ENB0 показывает результаты не намного хуже ENB3, но быстрее учится
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB0(
    weights='imagenet', include_top=False, input_shape=(size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
# строим нейросеть для анализа табличных данных
tabular_model = Sequential([
    L.Input(shape=X.shape[1]),
    L.Dense(512, activation='relu'),
    L.Dropout(0.5),
    L.Dense(256, activation='relu'),
    L.Dropout(0.5),
])

In [ ]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
])

In [ ]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate(
    [efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input,
              tabular_model.input, nlp_model.input], outputs=head)
# model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list
                    )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:, 0]
sample_submission.to_csv('nn3_submission.csv', index=False)

In [ ]:
blend_sub_predict_3 = (
    np.exp(submissions_cat['blend'].values) + sub_predict_nn3[:, 0]) / 2
sample_submission['price'] = blend_sub_predict_3
sample_submission.to_csv('blend_submission_3.csv', index=False)
sample_submission.head(10)

#### mape на лидерборде 11.87%, что хуже, чем предыдущие версии blend. Возможно, для обработки изображений были выбраны неправильные аугментации